In [9]:
import cv2 as cv
import numpy as np

In [10]:
cap = cv.VideoCapture(0)

In [11]:
def nothing(x):
    pass

In [12]:
cv.namedWindow('adjust values')
cv.resizeWindow('adjust values',(300,300))
cv.createTrackbar('thresh','adjust values',0,255,nothing)
cv.createTrackbar('h_min','adjust values',0,255,nothing)
cv.createTrackbar('h_max','adjust values',255,255,nothing)
cv.createTrackbar('sat_min','adjust values',0,255,nothing)
cv.createTrackbar('sat_max','adjust values',255,255,nothing)
cv.createTrackbar('val_min','adjust values',0,255,nothing)
cv.createTrackbar('val_max','adjust values',255,255,nothing)

while True:
    ret,frame = cap.read()
    if ret:
        frame = cv.resize(frame,(400,400))
        hsv = cv.cvtColor(frame,cv.COLOR_BGR2HSV)
        h_min = cv.getTrackbarPos('h_min','adjust values')
        h_max = cv.getTrackbarPos('h_max','adjust values')
        sat_min = cv.getTrackbarPos('sat_min','adjust values')
        sat_max = cv.getTrackbarPos('sat_max','adjust values')
        val_min = cv.getTrackbarPos('val_min','adjust values')
        val_max = cv.getTrackbarPos('val_max','adjust values')
        threshold_value = cv.getTrackbarPos('thresh','adjust values')
        lower = np.array([h_min,sat_min,val_min])
        upper = np.array([h_max,sat_max,val_max])
        mask = cv.inRange(hsv,lower,upper)
        filter = cv.bitwise_and(frame,frame,mask=mask)
        # as we know normally the mask create white background and black object
        # but for contours we require its reverse so we will now apply bitwise_not
        # to reverse the mask result
        mask = cv.bitwise_not(mask)
        _,thresh = cv.threshold(mask,threshold_value,255,cv.THRESH_BINARY)
        dilate = cv.dilate(thresh,(1,1),iterations=6)
        cntrs,hierarchy = cv.findContours(dilate,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
        frame = cv.drawContours(frame,cntrs,-1,(0,255,0),3)    
        cv.imshow('contours',frame)
        cv.imshow('mask',mask)
        cv.imshow('filter',filter)
        cv.imshow('thresh',thresh)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    else:
        break
cap.release()
cv.destroyAllWindows()